### Fleet Sales Pricing @ Fjord Motor, Columbia Business School

In [2]:
#Package imports
import pandas as pd
import numpy as np
import math
from scipy.optimize import minimize

In [3]:
#Data loading
MSRP = 25000
cost = 15000
all_data = pd.read_excel('/Users/user/Desktop/FjordMotor.xls',sheet_name = "Sheet2")
all_data['P'] = all_data['Unit_Price']/MSRP
all_data.Units = all_data.Units.astype('float')
all_data.Unit_Price = all_data.Unit_Price.astype('float')
all_data.Win = all_data.Win.astype('float')
all_data

,Bid,Units,Unit_Price,Total_Price,Win,Discount,Unit_Marg,Units_Sold,Tot_Margin,P
0,1,12.0,16551.0,198612,1.0,0.33796,1551,12,18612,0.66204
1,2,24.0,16272.0,390523,0.0,0.34912,1272,0,0,0.65088
2,3,16.0,21266.0,340261,1.0,0.14936,6266,16,100256,0.85064
3,4,21.0,18805.0,394910,0.0,0.24780,3805,0,0,0.75220
4,5,27.0,15884.0,428864,0.0,0.36464,884,0,0,0.63536
...,...,...,...,...,...,...,...,...,...,...
3995,3996,10.0,15316.0,153162,1.0,0.38736,316,10,3160,0.61264
3996,3997,17.0,21131.0,359232,1.0,0.15476,6131,17,104227,0.84524
3997,3998,25.0,18005.0,450125,1.0,0.27980,3005,25,75125,0.72020
3998,3999,14.0,20395.0,285533,1.0,0.18420,5395,14,75530,0.81580


In [4]:
#Data splitting
police_data = all_data.iloc[:2000,:] #bids 1 through 2000
corporate_data = all_data.iloc[2000:,:] #bids 2001 through 4000

#### Question 1: Find a 2-param logistic model that best estimates the prob winning each bid....

In [5]:
#Part i) What are the values of a and b that maximize the sum of log likelihoods?

#Defining a func that calculates the probability of a winning bid
def winning_prob_func(a,b,p): #a = intercept, b = beta, p = Unit_Price/MSRP
    winning_prob = 1/(1+np.exp(a+b*p))
    return winning_prob

#Defining a func that sums log likelihoods
def log_likelihood_sum_func(a,b,args):
    p, y = args
    winning_prob = winning_prob_func(a,b,p)
    log_likelihood_sum = np.sum(np.log(np.power(winning_prob,y) * np.power(1-winning_prob,1-y)))
    return log_likelihood_sum

#Defining a negative func for summing log likelihoods (for use with scipy.minimize)
def neg_log_likelihood_sum_func(param, *args):
    a, b = param
    neg_log_likelihood_sum = -log_likelihood_sum_func(a, b, args=args)
    return neg_log_likelihood_sum

#Defining a func to maximize the sum of log likelihoods by minimizing the negative sum of log likelihoods
def max_log_likelihood_func(args):
    initial_guess = np.array([0,0])
    results = minimize(neg_log_likelihood_sum_func, x0 = initial_guess, args = args)
    optimal_a, optimal_b = results.x
    return round(optimal_a,2), round(optimal_b,2)

#Calculating values of a and b
args = (all_data['P'],all_data['Win'])
optimal_a, optimal_b = max_log_likelihood_func(args)
print("The values of a and b that maximize the sum of log likelihoods are:", "a =",optimal_a,"& b =",optimal_b)

The values of a and b that maximize the sum of log likelihoods are: a = -7.76 & b = 9.16


In [7]:
#ii) What is the optimum price Fjord should offer, assuming it is going to offer a single price for each bid?

#Defining a function to calculate expected margin
def expected_margin_func(price, args):
    a, b = args
    p = price/MSRP
    winning_prob = winning_prob_func(a,b,p)
    expected_margin = (price - cost) * winning_prob
    return expected_margin

#Defining a negative func (for scipy minimize)
def neg_expected_margin_func(param, *args):
    price = param
    neg_expected_margin = -expected_margin_func(price, args)
    return neg_expected_margin

#Maximizing expected margin by minimizing negative expected margin
def maximize_expected_margin_func(args):
    initial_guess = np.array([0])
    results = minimize(neg_expected_margin_func, x0=initial_guess, args=args)
    optimal_price = results.x[0]
    return round(optimal_price,2)

#Calculating optimum price
args = (optimal_a, optimal_b)
optimal_price = maximize_expected_margin_func(args)
print("The optimal single price is $", optimal_price)

The optimal single price is $ 20830.48


In [11]:
#iii) What would the expected total contribution be for 4000 bids? 

#Defining func to calculate total expected contrbution
def total_expected_contribution_func(units,price,cost,a,b):
    p = optimal_price/MSRP
    expected_total_contribution = np.sum(units * (optimal_price - cost) * winning_prob_func(a,b,p))
    return round(expected_total_contribution,2)
                                         
#Calculating result
expected_total_contribution = total_expected_contribution_func(all_data['Units'],optimal_price,cost,optimal_a,optimal_b)
print("The expected total contribution is $", expected_total_contribution)

#Defining func to calculate actual total contribution
def total_actual_contribution_func(wins,units,unit_prices):
    actual_total_contribution = np.sum(wins * units * (unit_prices - cost))
    return round(actual_total_contribution,2)

#Calculating result
actual_total_contribution = total_actual_contribution_func(all_data['Win'],all_data['Units'],all_data['Unit_Price'])
print("The total actual contribution was $", actual_total_contribution)
print("The optimal pricing params result in an improvement of $",expected_total_contribution-actual_total_contribution,
     "or a", (expected_total_contribution/actual_total_contribution-1)*100, "% improvement")

The expected total contribution is $ 241901569.6
The total actual contribution was $ 171829002.0
The optimal pricing params result in an improvement of $ 70072567.6 or a 40.78040772185827 % improvement


#### Question 2: Police Departments & Corporate Buyers

In [16]:
#Part i) What are the optimum values of a & b for Police and Corporates?

#Police
args = (police_data['P'], police_data['Win'])
police_optimal_a, police_optimal_b = max_log_likelihood_func(args)
print("The values of a and b that maximize the sum of log likelihoods are (Police Sales):", "a =",police_optimal_a,"& b =",police_optimal_b)

#Corporates
args = (corporate_data['P'], corporate_data['Win'])
corporate_optimal_a, corporate_optimal_b = max_log_likelihood_func(args)
print("The values of a and b that maximize the sum of log likelihoods are (Corporate Sales):", "a =",corporate_optimal_a,"& b =",corporate_optimal_b)

The values of a and b that maximize the sum of log likelihoods are (Police Sales): a = -14.22 & b = 20.01
The values of a and b that maximize the sum of log likelihoods are (Corporate Sales): a = -27.88 & b = 28.81


In [24]:
#ii) What are the optimum prices Fjord should charge to the Police and Corporates?

#Police
args = (police_optimal_a, police_optimal_b)
police_optimal_price = maximize_expected_margin_func(args)
print("The optimum price for Police is $", police_optimal_price)

#Corporates
args = (corporate_optimal_a, corporate_optimal_b)
corporate_optimal_price = maximize_expected_margin_func(args)
print("The optimum price for Corporates is $", corporate_optimal_price)

The optimum price for Police is $ 17635.95
The optimum price for Corporates is $ 22436.49


In [25]:
#iii) What would the expected contribution be if Fjord had used these prices for the 4000 historic bids?

police_expected_contribution = total_expected_contribution_func(
    police_data['Units'],police_optimal_price,cost,police_optimal_a,police_optimal_b)

corporate_expected_contribution = total_expected_contribution_func(
    corporate_data['Units'], corporate_optimal_price,cost,corporate_optimal_a, corporate_optimal_b)

police_corporate_expected_contribution = police_expected_contribution + corporate_expected_contribution

print("The expected contribution from Police sales is $", police_expected_contribution)
print("The expected contribution from Corporate sales is $", corporate_expected_contribution)
print("The expected total contribution is $", police_corporate_expected_contribution)

The expected contribution from Police sales is $ 18151835.73
The expected contribution from Corporate sales is $ 221137268.38
The expected total contribution is $ 239289104.10999998


In [31]:
#iv) What is the difference between the contribution actually recieved and the best Fjord could do when it could not differentiate?
print("The optimal pricing params result in an improvement of $",police_corporate_expected_contribution-expected_total_contribution,
     "or a", (police_corporate_expected_contribution/expected_total_contribution-1)*100, "% improvement")

The optimal pricing params result in an improvement of $ -2612465.4900000095 or a -1.0799704583644876 % improvement


#### Question 3: New Model with Size of Order

In [32]:
#Part i) What is the resulting improvement in total log likelihood using the new model?

#Defining func for calculating winning probabilities (2 factor model)
def winning_prob_func_2(a,b,c,p,units):
    winning_prob = 1 / (1+np.exp(a+b*p+c*units))
    return winning_prob

#Defining a function for calculating the sum of log likelihoods 
def log_likelihood_sum_func_2(a,b,c,args):
    p, units, y = args
    winning_prob = winning_prob_func_2(a,b,c,p,units)
    log_likelihood_sum = np.sum(np.log(np.power(winning_prob,y) * np.power(1-winning_prob, 1-y)))
    return log_likelihood_sum

#Defining a negative func to minimize
def neg_log_likelihood_sum_func_2(param, *args):
    a, b, c = param
    negative_log_likelihood_sum = -log_likelihood_sum_func_2(a, b, c, args=args)
    return negative_log_likelihood_sum

#Defining a func to maximize the sum of log likelhoods
def max_log_likelihood_func_2(args):
    initial_guess = np.array([0,0,0])
    results = minimize(neg_log_likelihood_sum_func_2, x0 = initial_guess, args=args)
    optimal_a, optimal_b, optimal_c = results.x
    return round(optimal_a,2), round(optimal_b,2), round(optimal_c,2)

In [43]:
#Calculating values of a and b

#Previous model
args = (all_data['P'], all_data['Win'])
previous_log_likelihood_sum = log_likelihood_sum_func(optimal_a, optimal_b, args)

#Police
args = (police_data['P'],police_data['Units'],police_data['Win'])
police_optimal_a_2, police_optimal_b_2, police_optimal_c_2 = max_log_likelihood_func_2(args)
police_log_likelihood_2 = log_likelihood_sum_func_2(police_optimal_a_2,police_optimal_b_2,police_optimal_c_2,args)

#Corporate
args = (corporate_data['P'],corporate_data['Units'],corporate_data['Win'])
corporate_optimal_a_2, corporate_optimal_b_2, corporate_optimal_c_2 = max_log_likelihood_func_2(args)
corporate_log_likelihood_2 = log_likelihood_sum_func_2(corporate_optimal_a_2,corporate_optimal_b_2,corporate_optimal_c_2,args)

#Total
total_log_likelihood_sum = police_log_likelihood_2 + corporate_log_likelihood_2

print("The resulting improvement in log likelihood is $",total_log_likelihood_sum - previous_log_likelihood_sum,
     "or a", -1*(total_log_likelihood_sum/previous_log_likelihood_sum-1)*100, "% improvement")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


The resulting improvement in log likelihood is $ 1241.1297417747296 or a 54.87139535785066 % improvement


In [44]:
#Part ii) How does this compare with the improvement from differentiating police and corporate sales?

#Original model

#Police
args = (police_data['P'],police_data['Win'])
original_police_log_likelihood_sum = log_likelihood_sum_func(police_optimal_a,police_optimal_b,args)

#Corporate
args = (corporate_data['P'],corporate_data['Win'])
original_corporate_log_likelihood_sum = log_likelihood_sum_func(corporate_optimal_a,corporate_optimal_b,args)

#Total
total_original_log_likelihood_sum = original_police_log_likelihood_sum + original_corporate_log_likelihood_sum

#Comparison
print("The resulting improvement in log likelihood is $",total_original_log_likelihood_sum - previous_log_likelihood_sum,
     "or a", -1*(total_original_log_likelihood_sum/previous_log_likelihood_sum-1)*100, "% improvement")

The resulting improvement in log likelihood is $ 1231.6882052891074 or a 54.45397704625066 % improvement


In [45]:
#Part iii) What are the optimal prices Fjord should charge for orders of 20 cars and 40 cars to police and corporates?

#Defining a function to calculate expected margin
cost = 15000
def expected_margin_func_2(price,args):
    a,b,c,cost,units = args
    p = price/MSRP
    winning_prob = winning_prob_func_2(a,b,c,p,units)
    expected_margin = (price-cost)*winning_prob
    return expected_margin

#Defining a negative func to maximize using scipy minimize
def neg_expected_margin_func_2(param,*args):
    price = param
    negative_expected_margin = -expected_margin_func_2(price,args)
    return negative_expected_margin

#Defining a function to maximize expected margin (minimizing negative func)
def maximize_expected_margin_func_2(args):
    initial_guess = np.array([0])
    results = minimize(neg_expected_margin_func_2, x0=initial_guess, args=args)
    optimal_price = results.x[0]
    return round(optimal_price,2)

In [48]:
#Calculating results

#Police
police_20_args = (police_optimal_a_2, police_optimal_b_2, police_optimal_c_2, cost, 20)
police_20_optimal_price = maximize_expected_margin_func_2(police_20_args)
print("The optimal price for 20 cars sold to Police Departments is $", police_20_optimal_price)
police_40_args = (police_optimal_a_2, police_optimal_b_2, police_optimal_c_2, cost, 40)
police_40_optimal_price = maximize_expected_margin_func_2(police_40_args)
print("The optimal price for 40 cars sold to Police Departments is $", police_40_optimal_price)

#Corporates
corporate_20_args = (corporate_optimal_a_2, corporate_optimal_b_2, corporate_optimal_c_2, cost, 20)
corporate_20_optimal_price = maximize_expected_margin_func_2(corporate_20_args)
print("The optimal price for 20 cars sold to Corporates is $", corporate_20_optimal_price)
corporate_40_args = (corporate_optimal_a_2, corporate_optimal_b_2, corporate_optimal_c_2, cost, 40)
corporate_40_optimal_price = maximize_expected_margin_func_2(corporate_40_args)
print("The optimal price for 40 cars to Corporates is $", corporate_40_optimal_price)

The optimal price for 20 cars sold to Police Departments is $ 17659.85
The optimal price for 40 cars sold to Police Departments is $ 17289.99
The optimal price for 20 cars sold to Corporates is $ 22510.43
The optimal price for 40 cars to Corporates is $ 22664.51


In [49]:
#Part iv) Calculate optimal prices for all order sizes from 10 through 60 vehicles for both 
#police and corporate sales, and use these prices to determine the total contribution margin Fjord would 
#have received if it had used these prices in the 4,000 historic bids.

#Creating empty lists to append results to
police_optimal_prices = []
corporate_optimal_prices = []

#Iterating over all order sizes between 10 and 61
for i in range(10,61):
    #Police sales
    police_args = (police_optimal_a_2,police_optimal_b_2,police_optimal_c_2,cost,i)
    police_optimal_price = maximize_expected_margin_func_2(police_args)
    police_optimal_prices.append(police_optimal_price)
    #Corporate Sales
    corporate_args = (corporate_optimal_a_2,corporate_optimal_b_2,corporate_optimal_c_2,cost,i)
    corporate_optimal_price = maximize_expected_margin_func_2(corporate_args)
    corporate_optimal_prices.append(corporate_optimal_price)
    
#Appending results to a dataframe
pd.DataFrame(data={'Police Departments': police_optimal_prices,'Corporate Buyers': corporate_optimal_prices},index = range(10,61))

,Police Departments,Corporate Buyers
10,17864.35,22433.67
11,17843.42,22441.24
12,17822.53,22448.98
13,17801.76,22456.62
14,17781.07,22464.14
15,17760.59,22471.82
16,17740.18,22479.63
17,17719.93,22487.27
18,17699.65,22495.06
19,17679.68,22502.81


#### Question 4 Cornet Elizabeth

In [52]:
#How does the new price change the optimal price charged to police depts for 20 vehicle orders? For 40?
new_cost = 16000
police_20_args = (police_optimal_a_2, police_optimal_b_2, police_optimal_c_2, new_cost, 20)
police_20_optimal_price = maximize_expected_margin_func_2(police_20_args)
print("The new optimal price for 20 cars sold to Police Departments is $", police_20_optimal_price)
police_40_args = (police_optimal_a_2, police_optimal_b_2, police_optimal_c_2, new_cost, 40)
police_40_optimal_price = maximize_expected_margin_func_2(police_40_args)
print("The new optimal price for 40 cars sold to Police Departments is $", police_40_optimal_price)

The new optimal price for 20 cars sold to Police Departments is $ 18173.33
The new optimal price for 40 cars sold to Police Departments is $ 17884.73
